Ensuring Security of Cloud Server Instances
========================

@DanDye; MIL-OSS Hackathon, Tampa, FL October 23rd - 25th 2015

In [85]:
amis = [ {"AMI_ID": "ami-2df5b444", "AMI_Name": "rhel6.4-oracle-ami"},  # works
         {"AMI_ID": "ami-c7d379ae", "AMI_Name": "jboss-as-7.1.1.Final-as-a-service-RHEL6.3-64bit"},
         {"AMI_ID": "ami-dbb5c6b2", "AMI_Name": "NOW Developers RHEL6"},
       ]
AWS_KEYS_MILOSS="AWS_KEYS_MILOSS"
AWS_KEYS_MILOSS_FILE="/home/dandye/.ssh/AWS_KEYS_MILOSS.pem"
AWS_ID = open("/home/dandye/AWS_ID.txt").read().strip()
AWS_KEY = open("/home/dandye/AWS_KEY.txt").read().strip() # keeping secrets isn't easy
AWS_REGION = "us-east-1"

In [157]:
from boto import ec2
for i,ami in enumerate(amis[1:]):
    conn = ec2.connect_to_region(AWS_REGION,
                                 aws_access_key_id=AWS_ID,
                                 aws_secret_access_key=AWS_KEY)
    #conn.run_instances(ami["AMI_ID"], key_name="AWS_KEYS_MILOSS")

In [208]:
reservations = conn.get_all_reservations()

test_amis = {}

instances = [x.instances for x in reservations]

for instance in instances:
    if "running" in instance[0].state:
        print instance[0].image_id
        test_amis[instance[0].image_id] = {}
        test_amis[instance[0].image_id].update(instance[0].__dict__)
   
print [test_amis[akey]["_state"] for akey in test_amis.keys()]
# run the above cell until all states are 'running'

ami-2df5b444
[running(16)]


In [210]:
"""Setup some SSH/SFTP connex"""
import paramiko

mykey = paramiko.RSAKey.from_private_key_file("/home/dandye/.ssh/AWS_KEYS_MILOSS.pem")
sshuser = 'ec2-user' # FixMe: does this work for each AMI?

clients = []
for akey in test_amis.keys():
    public_dns_name = test_amis[akey]["public_dns_name"]
    print public_dns_name
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    client.load_system_host_keys()
    client.connect(public_dns_name, username=sshuser, pkey=mykey)
    clients.append(client)

#sftp = paramiko.SFTPClient.from_transport(client)

"""Use Paramiko to run remote commands on Rackspace/EC2 Instance"""
def run(client, cmd):
    transport = client.get_transport()
    session = transport.open_session()
    session.set_combine_stderr(True)
    session.get_pty()
    session.exec_command("sudo {}".format(cmd))
    stdin = session.makefile('wb', -1)
    stdout = session.makefile('rb', -1)
    for line in stdout.read().splitlines():        
        print line

ec2-54-157-250-199.compute-1.amazonaws.com


In [142]:
for client in clients:
    run(client, "sudo yum install -y openscap-utils scap-security-guide")
    run(client, "rpm -ql scap-security-guide")
    run(client, """oscap xccdf eval \
        --profile stig-rhel6-server-upstream \
        --results /tmp/ssg-results.xml \
        --report /tmp/ssg-results.html \
        --cpe /usr/share/xml/scap/ssg/content/ssg-rhel6-cpe-dictionary.xml \
              /usr/share/xml/scap/ssg/content/ssg-rhel6-xccdf.xml""") # fabric runs command on each host

In [211]:
#from fabric.api import env, run, get
#report = get("/tmp/ssg-results.html", "ami_{}_ssg_results.html".format(AMI_ID))

In [223]:
# SCP the report
for ami in test_amis.keys():
    print ami #ami = "ami-94a3eefc" # FixMe: hard-coded
    transport = paramiko.Transport((test_amis[ami]["public_dns_name"], 22))  # server is IP of box
    transport.connect(username="ec2-user", pkey=mykey)
    sftp = paramiko.SFTPClient.from_transport(transport)
    local_name = "{}_ssg_results".format(ami)
    sftp.get("/tmp/ssg-results.html", "{}.html".format(local_name))
    sftp.get("/tmp/ssg-results.xml", "{}.xml".format(local_name))
    test_amis[ami]["local_name"] = local_name

ami-2df5b444


In [228]:
import os
from IPython.display import IFrame
#render = lambda url: IFrame(url, width=900, height=500)

for ami in test_amis.keys():
    url = "{}.html".format(test_amis[ami]["local_name"])
    print url
    IFrame(url, width=900, height=500)
    break

ami-2df5b444_ssg_results.html


In [229]:
IFrame("ami-2df5b444_ssg_results.html", width=900, height=500)